In [132]:
import json
from nltk.corpus import wordnet as wn
from nltk.corpus import wordnet_ic
import nltk
from nltk.stem import WordNetLemmatizer, SnowballStemmer, PorterStemmer

nltk.download('wordnet_ic')
brown_ic = wordnet_ic.ic('ic-brown.dat')
from itertools import product

import os
import itertools
import re

from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter

import numpy as np

[nltk_data] Downloading package wordnet_ic to
[nltk_data]     /Users/viviancai/nltk_data...
[nltk_data]   Package wordnet_ic is already up-to-date!


# Match game with game keywords

In [271]:

# with open('keyphrases_and_keywords.json', 'r', encoding='utf8') as in_json_file:
#     common = json.load(in_json_file)


# with open('keywords_count_dict.json', 'r', encoding='utf8') as in_json_file:
#     word_count = json.load(in_json_file)

with open('movie_keyphrases_and_keywords.json', 'r', encoding='utf8') as in_json_file:
    common = json.load(in_json_file)


with open('movie_keywords_count_dict.json', 'r', encoding='utf8') as in_json_file:
    word_count = json.load(in_json_file)

# with open('.json', 'r', encoding='utf8') as aa:
#     aa = json.load(aa)

In [272]:
def clean_syns(syn):
    syn = syn.lower()
    cleaned = re.sub('[^A-Za-z]+', ' ', syn).strip()
    if cleaned!=syn:
        return []
    split = cleaned.split()
    split = list(filter(lambda x: len(x)>2, split))
    return split

def get_syns(word):
    res = set()
    syns = wn.synsets(word, pos=wn.NOUN)
    syns.extend(wn.synsets(word, pos=wn.ADJ))
    for sense in syns:
        if sense.pos()=='s':
            sense = sense.similar_tos()[0]
#         print(sense.lemma_names())
        cleaned_syns = list(map(clean_syns, sense.lemma_names()))
        cleaned_syns = list(itertools.chain(*cleaned_syns))
        res.update(cleaned_syns)
        
#         # Derived forms
#         derived = [n.name() for l in sense.lemmas() for n in l.derivationally_related_forms()]
#         derived = list(map(clean_syns, derived))
#         derived = list(itertools.chain(*derived))
#         res.update()
    if word in res:
        res.remove(word)
    return list(res)

    
def should_keep(word):
    if word not in word_count.keys():
        return False
    
    (word,pos) = nltk.pos_tag([word])[0]
    if pos[:2]=='NN' or pos[:2]=='JJ':
        return True
    return False

def add_weight(d):
    for word in list(d):
        count = d[word]
        (word,pos) = nltk.pos_tag([word])[0]
        if pos[:2]=='NN':
            d[word] = d[word]*2
            pos = 'n'
        elif pos[:2]=='JJ':
            d[word] = d[word]*1.5
            pos = 'a'
        
        derived_set = set()
        
        syns = wn.synsets(word, pos=pos)
        if pos=='n':
            for sense in syns:
                if sense.pos() == 's':
                    sense = sense.similar_tos()[0]
                derived = [n.name() for l in sense.lemmas() for n in l.derivationally_related_forms()]
                derived = list(map(clean_syns, derived))
                derived = list(itertools.chain(*derived))
                derived = list(filter(lambda x: nltk.pos_tag([x])[0][1][:2]=='JJ', derived))                
                derived_set.update(derived)

            for derived in derived_set:
                if derived not in d.keys():
                    d[derived] = d[word]*0.9
    return d

def process_info(info):
    lst = list(info['keywords'])
    lst = list(filter(should_keep, lst))
    
    ctr = add_weight(dict(Counter(lst)))
    
    numbers = np.array(list(ctr.values()))
    squared = numbers ** 2
    norm = np.sum(squared)
    norm = np.sqrt(norm)
#     numbers = ctr.values()['vector'].values()
#     print(numbers)
#     print(ctr)
    return ctr, norm
    
    
# keyword -> list of keywords and their synonyms
# keyphrase -> break down to words and for each word, check if its synonym in word_to_synphrase

In [273]:
info_path = 'movie_info/'

# appid_to_vec = dict()
link_to_vec = dict()
count = 0
for filename in os.listdir(info_path):
    if filename.endswith('.json'):
        with open(info_path+filename, 'r', encoding='utf8') as in_json_file:
            info = json.load(in_json_file)
            link = info['link']
            vector, norm = process_info(info)
            link_to_vec[link] = {'link':link, 'vector':vector, 'norm':norm}
    
    count+=1
    if count%100==0:
        print(count)
    if count == 1:
        print(link_to_vec)


{'flakes': {'link': 'flakes', 'vector': {'display': 2, 'bowl': 2, 'cable': 2, 'cool': 2, 'charm': 2, 'bargain': 2, 'comedy': 2, 'corn': 2, 'capitalism': 2, 'breezy': 2, 'dvd': 2, 'look': 2, 'matinee': 2, 'comical': 1.8, 'comic': 1.8, 'capitalistic': 1.8}, 'norm': 7.856207736560942}}
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13

In [274]:

# output_path = 'appid_to_vec.json'
# with open(output_path, 'w') as f:
#     f.write(json.dumps(appid_to_vec, indent=4)+'\n')

output_path = 'movielink_to_vec.json'
with open(output_path, 'w') as f:
    f.write(json.dumps(link_to_vec, indent=4)+'\n')

with open('phrase_word_to_synphrase.json', 'r', encoding='utf8') as in_json_file:
    word_to_synphrase = json.load(in_json_file)

with open('inverse_keyword_phrases.json', 'r', encoding='utf8') as in_json_file:
    inverse = json.load(in_json_file)



In [269]:
# input_tags = ['adventure','great story','graphic','open world','soundtrack']
# input_tags = ['romance','love story','graphic','open world','soundtrack']
input_tags = ['game', 'world-war', 'geralt canon', 'open world', 'friends']

def clean_input(tags):
    lemmatizer = WordNetLemmatizer()
    def helper(tag):
        tag = tag.lower()
        tag = re.sub('[^A-Za-z]+', ' ', tag).strip()
        if len(tag.split())==1:
            (tag,pos) = nltk.pos_tag([tag])[0]
            if pos == 'n' or pos == 'a':
                tag = lemmatizer.lemmatize(tag, pos)
            else:
                tag = lemmatizer.lemmatize(tag, 'n')
        return tag
    tags = list(map(helper, tags))
    
    stopwords = ['game','play','video']
    tags = list(filter(lambda x:x not in stopwords, tags))
    return tags
    

def process_input_match_phrase(tags):
    res = dict()
    
    phrases = list(filter(lambda x: len(x.split())>1, tags))
    
    words = list(map(lambda x: x.split(), input_tags))
    words = list(itertools.chain(*words))
    
    word_to_tag = dict()
    for word in words:
        for tag in tags:
            if word in tag:
                t = word_to_tag.get(word, list())
                t.append(tag)
                word_to_tag[word] = t
    
    for phrase in phrases:
        if phrase in common['keyphrases']:
            res[phrase] = 3
    
    phrases.remove(phrase)
    
    for word in words:
        (word,pos) = nltk.pos_tag([word])[0]
        if pos=='n':
            synphrases = word_to_synphrase.get(word, list())
            for synphrase in synphrases:
                res[word_to_tag[word]] = 1.5
    
    game_to_score = dict()
    game_to_phrases = dict()
    for k in res.keys():
        games = inverse.get(k, list())
        for game in games:
            score = game_to_score.get(game, 0)
            score += res[k]
            phrases = game_to_phrases.get(game, set())
            phrases.add(k)
            game_to_score[game] = score
            game_to_phrases[game] = phrases
    
    final = {game: (game_to_score[game], game_to_phrases[game]) for game in game_to_score.keys()}
    final = [(game, score_phrases) for (game, score_phrases) in final.items()]
    final = sorted(final, key=lambda x:(-x[1][0], -len(x[1][1]), x[0]))
    return final

input_tags = clean_input(input_tags)
print(process_input_match_phrase(input_tags))
    
    
    
def process_input(tags):
    tags = list(map(lambda x: x.split(), input_tags))
    tags = list(itertools.chain(*tags))
    
    # Get syns for each tag
    tag_to_syns = dict()
    for tag in tags:
        syns = tag_to_syns.get(tag, list())
        syns.extend(get_syns(tag))
        tag_to_syns[tag] = syns
    
    # Get all the words to put in the final vector
    allwords = list(tags)
    for syns in tag_to_syns.values():
        allwords.extend(syns)
    
    ctr = Counter(allwords)
    ctr = dict(ctr)
    
    # Adjust weight
    for word in set(tags):
        count = ctr[word]
        (word,pos) = nltk.pos_tag([word])[0]
        if pos[:2]=='NN':
            if word in tags:
                ctr[word] = ctr[word]*2
            else:
                ctr[word] = ctr[word]*1.75
            pos = 'n'
        elif pos[:2]=='JJ':
            if word in tags:
                ctr[word] = ctr[word]*1.75
            else:
                ctr[word] = ctr[word]*1.5
            pos = 'a'
        else:
            continue
        
        derived_set = set()
        
        syns = wn.synsets(word, pos=pos)
        if pos=='n':
            for sense in syns:
                derived = [n.name() for l in sense.lemmas() for n in l.derivationally_related_forms()]
                derived = list(map(clean_syns, derived))
                derived = list(itertools.chain(*derived))
                derived = list(filter(lambda x: nltk.pos_tag([x])[0][1][:2]=='JJ', derived))                
                derived_set.update(derived)

            for derived in derived_set:
                if derived not in ctr.keys():
                    ctr[derived] = ctr[word]*0.9
    
    numbers = np.array(list(ctr.values()))
    squared = numbers ** 2
    norm = np.sum(squared)
    norm = np.sqrt(norm)
    
    return ctr, norm
    
input_vector, norm = process_input(input_tags)


# lemmatizer = WordNetLemmatizer()
# lemmatizer.lemmatize("romantic")

# stemmer  = SnowballStemmer('english')
# stemmer.stem('romantic')

# stemmer = PorterStemmer()
# stemmer.stem('romance')

[('226700', (6, {'open world', 'world war'})), ('312660', (6, {'open world', 'world war'})), ('378610', (6, {'open world', 'world war'})), ('50130', (6, {'open world', 'world war'})), ('10090', (3, {'world war'})), ('10150', (3, {'open world'})), ('102500', (3, {'open world'})), ('1030830', (3, {'open world'})), ('1030840', (3, {'open world'})), ('1055540', (3, {'open world'})), ('105600', (3, {'open world'})), ('1056640', (3, {'open world'})), ('1056960', (3, {'open world'})), ('107410', (3, {'open world'})), ('1085660', (3, {'open world'})), ('108710', (3, {'open world'})), ('108800', (3, {'open world'})), ('1097840', (3, {'open world'})), ('110800', (3, {'open world'})), ('1116960', (3, {'open world'})), ('1121560', (3, {'open world'})), ('1128000', (3, {'open world'})), ('113400', (3, {'open world'})), ('11450', (3, {'open world'})), ('1151340', (3, {'open world'})), ('1151640', (3, {'open world'})), ('115320', (3, {'open world'})), ('1172380', (3, {'open world'})), ('1172620', (3,

In [267]:
def cos_sim(d1, d2, norm1, norm2):
    intersect = set(d1.keys()).intersection(set(d2.keys()))
    score = sum(d1[k]*d2[k] for k in intersect)
    return (score, intersect)

# cos_sim(input_vector, appid_to_vec['47810']['vector'])
# print(input_vector)
# print(appid_to_vec['47810']['vector'])

res = list(map(lambda x: (x['app_id'], cos_sim(input_vector, x['vector'], norm, x['norm'])), appid_to_vec.values()))
res = sorted(res, key=lambda x:(-x[1][0], x[0]))
res

[('24980',
  (25.665,
   {'account',
    'active',
    'graphic',
    'level',
    'romance',
    'romantic',
    'soundtrack',
    'story',
    'universe'})),
 ('901242',
  (25.665,
   {'account',
    'active',
    'graphic',
    'level',
    'romance',
    'romantic',
    'soundtrack',
    'story',
    'universe'})),
 ('17460',
  (24.665,
   {'active',
    'available',
    'graphic',
    'history',
    'level',
    'narrative',
    'romance',
    'romantic',
    'story',
    'universe'})),
 ('204450',
  (23.925,
   {'active',
    'graphic',
    'history',
    'level',
    'narration',
    'narrative',
    'reality',
    'soundtrack',
    'story',
    'tale'})),
 ('238010',
  (23.665,
   {'active',
    'earth',
    'global',
    'graphic',
    'level',
    'mankind',
    'soundtrack',
    'story',
    'universe'})),
 ('278080',
  (23.665,
   {'active',
    'graphic',
    'history',
    'level',
    'romance',
    'romantic',
    'soundtrack',
    'story'})),
 ('260230',
  (23.165,
   

# Match keyphrase directly

In [207]:
with open('phrase_word_to_synphrase.json', 'r', encoding='utf8') as in_json_file:
    word_to_synphrase = json.load(in_json_file)


In [211]:
# input_tags = ['adventure','great story','graphic','open world','soundtrack']
# input_tags = ['romance','love story','graphic','open world','soundtrack']
input_tags = ['game', 'world war', 'great story', 'open world']

input_phrase = list(filter(lambda x: len(x.split())>1, input_tags))

input_phrase

['world war', 'great story']

In [275]:
movielink_to_wordsphrases = dict()

movie_info_path='movie_info/'

for filename in os.listdir(movie_info_path):
    if filename.endswith('.json'):
        with open(movie_info_path+filename, 'r', encoding='utf8') as in_json_file:
            print("Process file", filename)
            info = json.load(in_json_file)
            movielink_to_wordsphrases[info['link']] = info



Process file info_flakes.json
Process file info_jack_ryan_shadow_recruit.json
Process file info_my_dog_skip.json
Process file info_1206664-free_style.json
Process file info_two_weeks.json
Process file info_the_crash_reel_2013.json
Process file info_star_trek_ii_the_wrath_of_khan.json
Process file info_my_blind_brother.json
Process file info_honkytonk_man.json
Process file info_1131721-punisher.json
Process file info_film_unfinished.json
Process file info_armageddon.json
Process file info_gigante.json
Process file info_memoir_of_war.json
Process file info_daylight.json
Process file info_bookwars.json
Process file info_hellboy_2_the_golden_army.json
Process file info_jimi_all_is_by_my_side.json
Process file info_full_contact.json
Process file info_cookout.json
Process file info_the_valley_2018.json
Process file info_imagine.json
Process file info_farewell.json
Process file info_les_girls.json
Process file info_my-girl-2.json
Process file info_10008954-untraceable.json
Process file info_t

Process file info_belle_de_jour.json
Process file info_1036553-hitman.json
Process file info_eagle_eye.json
Process file info_1202807-easy_virtue.json
Process file info_10x10.json
Process file info_rock_rubber_45s.json
Process file info_the_canterville_ghost_1944.json
Process file info_foxy_brown.json
Process file info_rocky_iv.json
Process file info_sleepwalkers.json
Process file info_1010673-invaders_from_mars.json
Process file info_1074374-one_fine_day.json
Process file info_holmes_and_watson_2018.json
Process file info_man_with_two_brains.json
Process file info_the_last_survivors.json
Process file info_i_spy.json
Process file info_microcosmos.json
Process file info_csa_the_confederate_states_of_america.json
Process file info_coco_before_chanel.json
Process file info_for_those_in_peril_2014.json
Process file info_rugrats_go_wild.json
Process file info_sherpa.json
Process file info_crime_spree.json
Process file info_1004567-commando.json
Process file info_exhibition_2014.json
Process

Process file info_a_walk_into_the_sea_danny_williams_and_the_warhol_factory.json
Process file info_fame_ish.json
Process file info_bedtime_stories.json
Process file info_mfkz.json
Process file info_1005512-death_in_venice.json
Process file info_arthur_2_on_the_rocks.json
Process file info_alice_sweet_alice.json
Process file info_1059177-lenfer.json
Process file info_my_son_my_son_what_have_ye_done.json
Process file info_a_touch_of_sin.json
Process file info_the_two_of_us_1967.json
Process file info_i_am_heath_ledger.json
Process file info_1074022-lone_star.json
Process file info_and_while_we_were_here.json
Process file info_stand_up_guys.json
Process file info_training_day.json
Process file info_1216067-fifty_dead_men_walking.json
Process file info_boy_meets_girl_2014.json
Process file info_2050.json
Process file info_trash_2015.json
Process file info_hit_so_hard_2012.json
Process file info_journey_to_the_west_2014.json
Process file info_ted_2012.json
Process file info_larry_crowne.jso

Process file info_this_divided_state.json
Process file info_the_zen_of_bennett.json
Process file info_the_science_of_sleep.json
Process file info_murder_at_the_vanities.json
Process file info_brimstone_2017.json
Process file info_world_of_the_dead_the_zombie_diaries_2.json
Process file info_the_invitation.json
Process file info_monstrous.json
Process file info_27_dresses.json
Process file info_fresh_dressed.json
Process file info_flying_down_to_rio.json
Process file info_the_iceman_2013.json
Process file info_god_sex_and_apple_pie_1999.json
Process file info_wise_guys.json
Process file info_picnic_at_hanging_rock.json
Process file info_tai_chi_hero_2012.json
Process file info_darkon.json
Process file info_have_a_good_trip_adventures_in_psychedelics.json
Process file info_jack_goes_boating.json
Process file info_bad_ass_2012.json
Process file info_freejack.json
Process file info_1208926-vinyan.json
Process file info_dont_say_a_word.json
Process file info_the_program.json
Process file in

Process file info_divines.json
Process file info_client.json
Process file info_alone_2018.json
Process file info_oldboy_2013.json
Process file info_first_kill_2017.json
Process file info_the_sasquatch_gang.json
Process file info_hallowed_ground_2019.json
Process file info_gods_not_dead.json
Process file info_london_town_2016.json
Process file info_edc_2013_under_the_electric_sky.json
Process file info_whats_in_a_name_2013.json
Process file info_my_best_friends_wedding.json
Process file info_1061812-little_princess.json
Process file info_nightlight_2015.json
Process file info_queen_and_country.json
Process file info_todd_glass_act_happy.json
Process file info_herbie_hancock_possibilities.json
Process file info_everybody_loves_somebody.json
Process file info_c_word_2016.json
Process file info_bestiaire_2012.json
Process file info_brotherhood_2010.json
Process file info_loved_one.json
Process file info_crossing_the_line_2007.json
Process file info_10010458-proposal.json
Process file info_

...

Process file info_eye_for_an_eye_1996.json
Process file info_raiders_the_story_of_the_greatest_fan_film_ever_made.json
Process file info_our_man_in_tehran.json
Process file info_alex_rider_operation_stormbreaker.json
Process file info_perfect_man.json
Process file info_motherhood.json
Process file info_written_in_blood.json
Process file info_blue_story.json
Process file info_animals_united.json
Process file info_tyler_perrys_good_deeds.json
Process file info_twitches.json
Process file info_entre_nos.json
Process file info_killers_2010.json
Process file info_the_woman_in_black_2_angel_of_death.json
Process file info_10009828-house_of_sleeping_beauties.json
Process file info_baadasssss.json
Process file info_the_adventurers_2017.json
Process file info_johnny_guitar.json
Process file info_harry_potter_and_the_chamber_of_secrets.json
Process file info_marc_maron_too_real.json
Process file info_the_fog_of_war_eleven_lessons_from_the_life_of_robert_s_mcnamara.json
Process file info_riot_in_c

Process file info_yours_mine_and_ours.json
Process file info_uninvited_2009.json
Process file info_millennium_actress_2001.json
Process file info_next.json
Process file info_the_europeans.json
Process file info_gemma_bovery.json
Process file info_capricorn_one.json
Process file info_slay_the_dragon.json
Process file info_khatta_meetha_2010.json
Process file info_david_cross_making_america_great_again.json
Process file info_parenthood.json
Process file info_presidents_last_bang.json
Process file info_the_voices.json
Process file info_oscar_and_lucinda.json
Process file info_north_dallas_forty.json
Process file info_private_life_of_henry_viii.json
Process file info_alan_partridge.json
Process file info_51_birch_street.json
Process file info_dark_horse_2011.json
Process file info_septic_man_2014.json
Process file info_fort_bliss.json
Process file info_queen_of_the_sun_what_are_the_bees_telling_us_2010.json
Process file info_private_peaceful_2012.json
Process file info_1112549-crossroads.j

Process file info_billu-barber.json
Process file info_10009799-caller.json
Process file info_feardotcom_2002.json
Process file info_hunt_for_the_wilderpeople.json
Process file info_dead_and_buried.json
Process file info_1211577-skeptic.json
Process file info_dr_bronners_magic_soapbox.json
Process file info_the_last_dragon.json
Process file info_viceroys_house.json
Process file info_walkabout.json
Process file info_charlie_brown_christmas.json
Process file info_rainbow_time.json
Process file info_superfly_2018.json
Process file info_how_do_you_know.json
Process file info_giant_little_ones.json
Process file info_winning_the_racing_life_of_paul_newman.json
Process file info_winnie_the_pooh_2011.json
Process file info_1194818-encounters_at_the_end_of_the_world.json
Process file info_nothing_but_the_blood.json
Process file info_when_i_saw_you.json
Process file info_tammy_2014.json
Process file info_used_cars.json
Process file info_la_terra_trema.json
Process file info_mimzy.json
Process fil

Process file info_realive.json
Process file info_city_of_women.json
Process file info_instinct_de_mort.json
Process file info_the_big_wedding_2013.json
Process file info_all_i_wanna_do.json
Process file info_mo_better_blues.json
Process file info_a_conspiracy_of_faith_2016.json
Process file info_the_man_who_knew_too_much.json
Process file info_aardvark_2018.json
Process file info_until_the_light_takes_us.json
Process file info_wishmaster_the_prophecy_fulfilled.json
Process file info_bad_taste.json
Process file info_cop_out.json
Process file info_baby_driver.json
Process file info_klitschko.json
Process file info_i_got_the_hookup.json
Process file info_disappearance_of_garcia_lorca.json
Process file info_signal.json
Process file info_the_incredible_mr_limpet_1964.json
Process file info_neds.json
Process file info_the_believers_1987.json
Process file info_this_is_not_berlin.json
Process file info_pavarotti.json
Process file info_hacksaw_ridge.json
Process file info_ip_man_2_legend_of_the

Process file info_parental_guidance_2012.json
Process file info_shooting_robert_king.json
Process file info_1112980-perfume.json
Process file info_bruno.json
Process file info_louise_bourgeois_the_spider_the_mistress_and_the_tangerine.json
Process file info_the_whole_truth_2016.json
Process file info_1079025-most_wanted.json
Process file info_enemy_at_the_gates.json
Process file info_greetings_from_tim_buckley.json
Process file info_dying_gaul.json
Process file info_exterminator.json
Process file info_1037177-deceived.json
Process file info_cartel_land.json
Process file info_puerto_ricans_in_paris.json
Process file info_1090492-war_at_home.json
Process file info_deja_vu.json
Process file info_the_lost_husband.json
Process file info_ice_station_zebra.json
Process file info_blood_glacier.json
Process file info_neighbors_2014.json
Process file info_the_heat.json
Process file info_cliffords_really_big_movie.json
Process file info_micmacs.json
Process file info_scandal_in_paris.json
Process

In [276]:
output_path = 'movielink_to_wordsphrases.json'
with open(output_path, 'w') as f:
    f.write(json.dumps(movielink_to_wordsphrases, indent=4)+'\n')

In [322]:
movie_dir = '/Users/viviancai/Desktop/cs4300/cs4300sp2021-cw887-qh75-rz92-yc687-yl698/data/movie_data/'
with open('movielink_to_wordsphrases.json', 'r', encoding='utf8') as in_json_file:
    movielink_to_reviews_wordsphrases = json.load(in_json_file)

with open(movie_dir+'movie_id_to_info.json', 'r', encoding='utf8') as in_json_file:
    movielink_to_info = json.load(in_json_file)
    
with open(movie_dir+'movie_desc_keywords.json', 'r', encoding='utf8') as in_json_file:
    movielink_to_desc_keywords = json.load(in_json_file)

FileNotFoundError: [Errno 2] No such file or directory: 'movielink_to_wordsphrases.json'

In [323]:
movie_info_dict = dict()
count = 0
for (link, basic_info) in movielink_to_info.items():
    info = {}
    info['name'] = basic_info['name']
    info['genre'] = basic_info['genre']
    info['content_rating'] = basic_info['content_rating']
    info['audience_count'] = basic_info['audience_count']
    
    if link in movielink_to_desc_keywords:
        info['desc_keywords'] = movielink_to_desc_keywords[link]
    else:
        info['desc_keywords'] = list()
    
    if link in movielink_to_reviews_wordsphrases:
        info['review_keywords'] = movielink_to_reviews_wordsphrases[link]['keywords']
        info['review_keyphrases'] = movielink_to_reviews_wordsphrases[link]['keyphrases']
    else:
        info['review_keywords'] = list()
        info['review_keyphrases'] = list()
    
    if count==0:
        print(movie_info_dict)
        count+=1
    movie_info_dict[link] = info


{}


In [295]:
output_path = 'movie_info.json'
with open(output_path, 'w') as f:
    f.write(json.dumps(movie_info_dict, indent=4)+'\n')

In [312]:
game_dir = '/Users/viviancai/Desktop/cs4300/cs4300sp2021-cw887-qh75-rz92-yc687-yl698/data/steamData/80k_data/'

with open(game_dir+'available_games.json', 'r', encoding='utf8') as in_json_file:
    available_games = json.load(in_json_file)

with open(game_dir+'id_to_info.json', 'r', encoding='utf8') as in_json_file:
    appid_to_info = json.load(in_json_file)
    
appid_to_info = dict(filter(lambda item: int(item[0]) in available_games, appid_to_info.items()))

with open(game_dir+'game_desc_keywords.json', 'r', encoding='utf8') as in_json_file:
    appid_to_desc_keywords = json.load(in_json_file)

with open('appid_to_vec.json', 'r', encoding='utf8') as in_json_file:
    appid_to_vec_norm = json.load(in_json_file)
    


In [313]:
len(appid_to_info)
len(available_games)

2736

In [319]:
game_info_dict = dict()

for appid in available_games:
    appid = str(appid)
    info = {}
    if appid not in appid_to_info:
        continue
    info['name'] = appid_to_info[appid]['name']
    info['developer'] = appid_to_info[appid]['developer']
    info['publisher'] = appid_to_info[appid]['publisher']
    info['tags'] = appid_to_info[appid]['tags']
    info['genre'] = appid_to_info[appid]['genre']
    info['num_players'] = appid_to_info[appid]['number_of_players']
    info['rating'] = appid_to_info[appid]['rating']
    info['mature_content'] = appid_to_info[appid]['mature_content']
    info['url'] = appid_to_info[appid]['url']
    
    if appid in appid_to_desc_keywords:
        info['desc_keywords'] = appid_to_desc_keywords[appid]
    else:
        info['desc_keywords'] = list()
    
    game_info_dict[appid] = info



In [320]:
output_path = 'game_info.json'
with open(output_path, 'w') as f:
    f.write(json.dumps(game_info_dict, indent=4)+'\n')